# 4.1 Shell Introduction

**Shells:** can both be an interactive program that interact with at the terminal, as well as an interpreter that can read from files that have shell commands in them to execute them.

The script file are just multiple commands together.  
The interpreter of the shebang is a design decision (portability vs. features).

Most majority of 2 shells: sh shell and bash shell  
"Bashisms": The features that's not defined on the POSIX (which is sh/POSIX shell).  

**Bash Ref Manual:** https://www.gnu.org/software/bash/manual/bash.html


# 4.2 Job Control

when running the command normally, it's going to be known as foregrounded. Take up the terminal and take terminal's attention (take the keyboard inputs of the terminal, and output it to the terminal).  

**Jobs table:** manage all the programs or commands that you've decided to Launch.  
`jobs`: retrieve currently running jobs that the shell is managing right now.  
`fg`: foreground a task, bring back the last thing that was suspended, not necessarily the last thing that was interacted with.  
`bg`: background a task, run the process at the background, but also letting the shell prompt for input.

If there are multiple jobs, then `fg` will resume the last thing that has suspended.  
`fg %<job num>`: resume the number of the job specified.  

`disown`: make the shell disown a process so it doesn't have to well manage it.


C/C++: return 0 or 1 is to report back to whoever launched this process, and what status does it had.  
- Exit status 0 is good/success/true
- Exit status not 0 is bad/failure/false

`echo $?`: get the value of exit status of last command  
to save the exit status, could immediately assign that value it to a variable, such as: `some_var=$?`

# 4.3 File Redirection

`>` output redirection, overwrite mode: `echo "hello world" > file.txt`  
It will erase whatever is in that file.

`>>` output redirection, append mode: `echo "dog cat" >> file.txt`  
It will append the content at the back.

`<` input redirection: `rev < file.txt`  
Recieve input from a file.

`2>` std error redirection: `status 2> err.txt`  
It will save the error message to a file.

`<<` Here document: `rev << LOL`  
Allow you to directly supply a large string as input to a command. It will end when encountered the keyword

`<<<` Here string (Bashism): `rev <<< "hello world"`  
Allow you to directly supply a string as input to a command.

`app 5<&3` copy file descriptor 3 to file descriptor 5  
`app 5<&3-` close file descriptor 3 afterwards  
`echo 1>&2` copy file descriptor 2 over to file descriptor 1

(Bashism): `echo "hello world" &> lol.txt`  
This allows messages stuff from both std output or std error at the same place.  
could also be applied with append mode `&>>`

Keep the error message but make the std output message to disappear:  
`./echo-err.sh > /dev/null`  

Redirect std err to memory hole as well:  
`./echo-err.sh > /dev/null 2> /dev/null`

# 4.4 Command Grouping

#### { commands; }
- groups commands together as a unit
- runs commands in current shell (that means variable set in command will able to access in shell)
- curly braces needs spaces around them (put a space after curly braces beginning and end)
- there needs to be a ; at the end of the command list

Example:  
`{ echo hello; echo world; echo dog; } > file.txt`



#### (commands;)
- group commands together as a unit
- runs commands in a **new shell instance** (subshell) (the assignment will be appearing outside of scope, inaccessable outside of command)
- doesn't require spaces around the parentheses

Example:  
`(echo hello; echo world; echo dog;) > file.txt`

# 4.5 Expansion

- Set a variable `cat=hello`
- Variable & string concatnation `echo goodbye$cat`
- Wrap around the variable name using `{}`: `echo ${cat}goodbye`
- inline set variable: `echo ${dog:-world}` (if the dog doesn't have a value, then dog=world)
- printout only if the variable is set: `echo ${dog:+world}`
- other operand: `=`, `?`

### Bash substring expansion (bashism)
`${varname:offset}`: print the value after the offset value.  
e.g. `dog=abcdefghijklmnop`, `echo ${dog:2}` will give `cdefghijklmnop`  

`${varname:offset:length}`: print "length" characters after the offset value  
e.g. `dog=abcdefghijklmnop`, `echo ${dog:2:}` will give `cdefg`  

### file text substituion/wildcards
e.g. find the file that starts with "tty" in /dev/:  
`ls -l /dev/tty*`

e.g. find the file that starts with "tty" and 1 unknown character in /dev/ (match 1 character):  
`ls -l /dev/ttyw?`

e.g. match a set of characters:  
`ls -l /dev/ttyw[13579]`  
(so it's either going to be ttyw1, ttyw3, ttyw5, ttyw7, ttyw9)

### Command substitution

Saving the commands into variables:  
`x=$(echo "hello" | rev)`, which basically means the rev of hello will be stored in variable x.  

Arithmetic expansion:  
`$((expression))`, e.g. `x=$((3*5))`, `echo $x` will output 15.

Representing x+=1: `x=$((x+1))`

### Process substitution (bashism)

See the difference between 2 files: `diff file1 file2`  

- `<` means it's the expected output
- `>` means it's the actual output

Syntax: `diff {expected_output} {actual_output}`

# 4.6 Quoting

**Escaping:** Using `\` to "escape" and do an alternate behavior  
e.g. `mkdir dir\ name\ with\ spaces`  

**Single Quotes:** preserve all characters  
e.g. `cd 'dir name with spaces'` or `echo 'my home is $HOME'` -> my home is $HOME  

**Double Quotes:** preserve all characters besides $, \\, and `` ` `` (allow for expansions!)  
e.g. `echo "my home is $HOME"` -> my home is /Users/lanceying  

Combining with escaping and double quotes: `echo "my home is \"$HOME\""` -> my home is "/Users/lanceying"  
`"my home is '$HOME'"` -> my home is '/Users/lanceying'  

To learn more: IFS (interfield separator)

# 4.7 Control Flow

### Conditional Tests

The condition will run based on the exit status.  
e.g. `if true; then echo "success"; fi`  

`test` could be used to check on things  
e.g. `test 5 -lt 3` and `$?` -> 1, which means that it's false and the exit status is 1.  
e.g. `test 1 -lt 4` and `$?` -> 0, which means the expression evaluated as true and the exit status is 0.  
Individual tests can wrapped together in a bracket: `[ 1 -lt 4 ]` or `[ 5 -lt 4 ]`, or `if [ 1 -lt 4 ]; then echo "correct"; fi`

Combinational tests:  
Note: whitespace is meaningful here.

In [ ]:
x=3
if [ $x -lt 4 ]; then echo "correct"; else echo "incorrect"; fi

x=7
if test $x -lt 4 ; then echo "correct"; else echo "incorrect"; fi

y=10
if [ $x -lt 4 ] && [ $y -gt 6 ]; then echo "yay"; fi

x=3
y=7
if ! [ $x -lt 4 ]; then echo "yay"; fi

In [ ]:
#!/bin/sh
if test-commands; then
    commands
elif more-test-commands; then
    more-commands
else
    alt-commands
fi

Bashism commands:  
e.g.  
`a="hello"` `b="goodbye"`  
`if [[ $a > $b ]]; then echo "correct"; fi` 

`[[]]` it's used for string comparisons, so:

`a=100` `b=2`  
`if [[ $a > $b ]]; then echo "correct"; fi` -> false, since 1 is before 2 in string order.  
For arithmetic comparisons: `if [[ $a -gt $b ]]; then echo "correct"; fi` -> true or `if (( $a > $b )); then echo "correct"; fi`

### While loop
`while <conditional-commands>; do`  
`done` -> terminate while loop  

`until <conditional-commands>; do`  
`done` -> until the conditional-command succeed

In [ ]:
#!/bin/sh

while test-commands; do
    commands
done

until test-commands; do
    commands
done

### For loop
It's like a for each loop, where given a set and iterating items inside it.  
e.g. `for var in list; do`  
`done`, where the list is basically just some space delimited sequence of words or strings  

`for x in 1 2 3 4; do echo $x; done`  
output:  
1  
2  
3  
4  

`for x in $(seq 1 10); do echo hello$x; done`  
output:  
hello1  
hello2  
hello3  
hello4  
hello5  
hello6  
hello7  
hello8  
hello9  
hello10  

In [ ]:
#!/bin/sh

for var in list; do
    commands
done

### Case statement
value: can be expanded variables here as well  
string patterns: setting up string patterns to match against  
command: the command to be executed  
multpat: could also match multiple patterns, i.e. (pattern1 or pattern2)  
*: default case

In [ ]:
#!/bin/sh
case value in
    pattern1 ) commands1 ;;
    pattern2 ) commands2 ;;
    multpat1 | multpat2 ) commands3 ;;
    * ) commands
esac

### Functions

In [ ]:
#!/bin/sh

function-name () {
    echo hello world
}

function-name2 () if [ x -eq 0 ]; then echo "hello"

function function-name3 () compound-command #bashism

Note: Functions work like mini shell-scripts: arguments work the exact same way with the @, #, and number variables  
Arguments don't have to be declared like in other languages